In [1]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
lr = 1e-3
env = gym.make('CartPole-v0')
env.reset()
goal_steps = 500
score_requirement = 50
initial_games = 10000

[2017-09-02 15:25:26,995] Making new env: CartPole-v0


In [3]:
def some_random_games_first():
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
            env.render()
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if done:
                break
                
# some_random_games_first()

In [6]:
# To get training data
def initial_population():
    training_data = []
    scores = []
    accepted_scores = []
    for _ in range(initial_games):
        score = 0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            action = random.randrange(0, 2)
            observation, reward, done, info = env.step(action)
            
            if len(prev_observation) > 0:
                game_memory.append([prev_observation, action])
                
            prev_observation = observation
            score += reward
            if done:
                break
        
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                    
                elif data[1] == 0:
                    output = [1, 0]
                    
                training_data.append([data[0], output])
                
        env.reset()
        scores.append(score)
    training_data_save = np.array(training_data)
    np.save('saved.npy', training_data_save)
    print('Average accepted score: ', mean(accepted_scores))
    print('Median accepted score: ', median(accepted_scores))
    print(Counter(accepted_scores))
    return training_data

In [7]:
initial_population()

[2017-09-02 15:36:03,949] You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.


Average accepted score:  61.74193548387097
Mean accepted score:  58.0
Counter({50.0: 36, 51.0: 32, 52.0: 26, 55.0: 20, 58.0: 20, 53.0: 19, 56.0: 19, 54.0: 18, 59.0: 15, 64.0: 12, 60.0: 12, 67.0: 10, 65.0: 10, 61.0: 9, 57.0: 9, 63.0: 8, 68.0: 7, 78.0: 7, 62.0: 7, 71.0: 6, 77.0: 6, 66.0: 5, 74.0: 5, 86.0: 4, 69.0: 4, 70.0: 4, 82.0: 4, 72.0: 3, 73.0: 3, 85.0: 3, 81.0: 3, 76.0: 2, 87.0: 2, 89.0: 2, 93.0: 2, 84.0: 2, 97.0: 1, 99.0: 1, 103.0: 1, 117.0: 1, 95.0: 1, 88.0: 1, 100.0: 1, 124.0: 1, 104.0: 1, 94.0: 1, 101.0: 1, 80.0: 1, 105.0: 1, 75.0: 1, 79.0: 1, 98.0: 1})


[[array([-0.04123111,  0.17530937, -0.0041156 , -0.25973237]), [0, 1]],
 [array([-0.03772493,  0.37048983, -0.00931025, -0.55371057]), [0, 1]],
 [array([-0.03031513,  0.56574127, -0.02038446, -0.84931221]), [1, 0]],
 [array([-0.0190003 ,  0.37090316, -0.0373707 , -0.56310838]), [1, 0]],
 [array([-0.01158224,  0.17632498, -0.04863287, -0.28242932]), [1, 0]],
 [array([-0.00805574, -0.01807077, -0.05428146, -0.00547288]), [1, 0]],
 [array([-0.00841716, -0.21237393, -0.05439092,  0.26960166]), [0, 1]],
 [array([-0.01266463, -0.01651968, -0.04899888, -0.03972831]), [1, 0]],
 [array([-0.01299503, -0.21090599, -0.04979345,  0.2371013 ]), [0, 1]],
 [array([-0.01721315, -0.01510933, -0.04505142, -0.0708629 ]), [1, 0]],
 [array([-0.01751533, -0.20955742, -0.04646868,  0.20727264]), [1, 0]],
 [array([-0.02170648, -0.40398515, -0.04232323,  0.48494255]), [0, 1]],
 [array([-0.02978619, -0.20829228, -0.03262438,  0.1792268 ]), [1, 0]],
 [array([-0.03395203, -0.40293256, -0.02903984,  0.46144198]), [